In [6]:
!pip install face_recognition

In [44]:
import os
from google.colab import drive
import face_recognition
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [46]:
unlabelled_folder = '/content/drive/MyDrive/unlabeled_library_images/' #this is the folder where images originally are before labeling
labelled_folder = '/content/drive/MyDrive/unlabeled_library_images/' #the images are moved here
spreadsheet_folder = '/content/drive/MyDrive/unlabeled_library_images/' #any database files go here

#note you can edit and put whatever folders you want those are just the name of my folders^

In [47]:
unlabelled_images = os.listdir(unlabelled_folder)
print(unlabelled_images)

['dwayne.jpg', '.ipynb_checkpoints', 'faces_data.csv', 'image_face_names.csv', 'new_data.csv']


In [48]:
if os.path.exists(spreadsheet_folder + 'faces_data.csv'):
    faces_data = pd.read_csv(spreadsheet_folder + 'faces_data.csv') #data of faces and encodings
else:
    faces_data = pd.DataFrame(columns = ['name','face_encoding'])
if os.path.exists(spreadsheet_folder + 'image_face_names.csv'):
    image_face_names = pd.read_csv(spreadsheet_folder + 'image_face_names.csv') #images and the existing names in those images
else:
    image_face_names = pd.DataFrame(columns = ['image_name','face_names'])
if os.path.exists(spreadsheet_folder + 'new_data.csv'):
    new_data = pd.read_csv(spreadsheet_folder + 'new_data.csv') #this is where the new labeling data is put
else:
    new_data = pd.DataFrame(columns = ['image_name','faces'])

In [49]:
faces_data.head()

,name\tface_encoding,name,face_encoding
0,NaN,Ashkar,"[-0.18324539065361023, 0.1227283924818039, 0.0..."
1,NaN,ikram,"[-0.08382032066583633, 0.08915513753890991, -0..."
2,NaN,khalil,"[-0.19419795274734497, 0.06705864518880844, 0...."
3,NaN,isamil,"[-0.14713019132614136, 0.05713728070259094, -0..."
4,NaN,Ebram,"[-0.07837033271789551, 0.04806605353951454, -0..."


In [50]:
image_face_names.head()

,image_name,face_names
0,WhatsApp Image 2024-05-30 at 10.18.47_97793ef2...,Ebram
1,WhatsApp Image 2024-07-17 at 18.59.31_81e74e82...,Ebram
2,321670358_1379018396174306_8323900925920846555...,Ebram
3,tom.jpg,Tom
4,dwayne.jpg,Dwayne


In [51]:
new_data.head()

,image_name,faces
0,dwayne.jpg,[dwayne]


In [52]:
  #df.loc[df['A'] == search_value, 'B'] = new_value example for later

In [ ]:
def find_closest_encoding(df, encoding):
    all_encodings = df['face_encoding'].apply(lambda x: np.array(x) if isinstance(x, list) else x)

    closest_distance = float('inf')
    closest_encoding = None
    closest_row_index = None
    for index, encodings in all_encodings.iteritems():
        distances = face_recognition.face_distance(encodings, encoding)
        min_distance = np.min(distances)
        if min_distance < closest_distance:
            closest_distance = min_distance
            closest_encoding = encodings[np.argmin(distances)]
            closest_row_index = index
    return closest_row_index, closest_encoding

In [59]:
for image in unlabelled_images:
  image_dir = unlabelled_folder+image
  try:
    loaded_image = face_recognition.load_image_file(image_dir)
    face_locations = face_recognition.face_locations(loaded_image)
    face_encodings = face_recognition.face_encodings(loaded_image, face_locations)
    face_encodings_sorted = [encoding for _, encoding in sorted(zip(face_locations, face_encodings), key=lambda x: x[0][3])] #sorted from left to right
    for face in face_encodings_sorted:

    print(new_data.loc[new_data['image_name'] == image,'faces'])

  except:
    print(f'{image} is not an image')



0    [dwayne]
Name: faces, dtype: object
.ipynb_checkpoints is not an image
faces_data.csv is not an image
image_face_names.csv is not an image
new_data.csv is not an image


newdata structure:
image_name, faces (from left to right)

Pseudocode for later:
```
for all_images:
  run face_recognition();
  for all_faces:
    if !smallest.face.distance == csv_face_name:
       add_encoding_to_faces_data();
    if new_face:
       add_row_to_faces_data();
    add_row_to_image_face_names(); //needs to follow face_encodings placement not the new_csv files placement
```

